In [1]:
import functools
import enum
import haiku as hk
import jax
import jax.numpy as jnp

/root/anaconda3/envs/mpnn/lib/python3.9/site-packages/haiku/_src/data_structures.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))


In [2]:
from BH.data_loader import *
from BH.generate_data import *
from Model_e import Model_e,Direction,Reduction
# from Load_Data import batch
from Train import train,print_accuracies

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
#@title Load data

#@markdown Training this model is pretty slow - an hour or so on the free tier colab, but subject to inactivity timeouts and pre-emptions.

#@markdown In order to make it possible to recreate the results from the paper reliably and quickly, we provide several helpers to either speed things up, or reduce the memory footprint:
#@markdown * Pretrained weights - greatly speeds things up by loading the trained model parameters rather than learning from the data
#@markdown * If you are running on a high memory machine (ie *not* on the free colab instance!) the input graph data can be loaded from a pickle (which is faster to load) and kept in memory (faster to re-use, but uses ~12Gb of memory). This makes no difference to training speed (it's only relevant for `generate_graph_data()` and `get_saliency_vectors()`).
DIR_PATH = "/Data/Ptab/n=5"


use_pretrained_weights = True  #@param{type:"boolean"}
hold_graphs_in_memory = False  #@param{type:"boolean"}

gb = 1024**3
total_memory = psutil.virtual_memory().total / gb
# Less than 20Gb of RAM means we need to do some things slower, but with lower memory impact - in
# particular, we want to allow things to run on the free colab tier.
if total_memory < 20 and hold_graphs_in_memory:
    raise RuntimeError(f"It is unlikely your machine (with {total_memory}Gb) will have enough memory to complete the colab's execution!")

print("Loading input data...")

full_dataset, train_dataset, test_dataset = load_input_data(DIR_PATH)

Loading input data...
Generating data from the directory /Data/Ptab/n=5


In [4]:
step_size = 0.001
batch_size = 16

num_classes = 2


model = Model_e(
    num_layers=3,
    num_features=64,
    num_classes=num_classes,
    direction=Direction.BOTH,
    reduction=Reduction.SUM,
    apply_relu_activation=True,
    use_mask=False,
    share=False,
    message_relu=True,
    with_bias=True)

loss_val_gr = jax.value_and_grad(model.loss)
opt_init, opt_update = optax.adam(step_size)

In [5]:
num_epochs = 10
trained_params = model.net.init(
    jax.random.PRNGKey(42),
    features=train_dataset.features[0],
    rows=train_dataset.rows[0],
    cols=train_dataset.columns[0],
    batch_size=1,
    edge_types=train_dataset.edge_types[0])
trained_opt_state = opt_init(trained_params)

2023-03-19 07:02:31.147246: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2023-03-19 07:02:31.163849: W external/org_tensorflow/tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2023-03-19 07:02:31.163888: W external/org_tensorflow/tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2023-03-19 07:02:31.189070: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:538] failed to load PTX text as a module: CUDA_ERROR_INVALID_IMAGE: device kernel image is invalid
2023-03-19 07:02:31.189117: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:543] error log buffer (63 bytes):

XlaRuntimeError: INTERNAL: Failed to execute XLA Runtime executable: run time error: custom call 'xla.gpu.func.launch' failed: Failed to load PTX text as a module: CUDA_ERROR_INVALID_IMAGE: device kernel image is invalid.

In [ ]:
for ep in range(1, num_epochs + 1):
    tr_data = list(
        zip(
            train_dataset.features,
            train_dataset.rows,
            train_dataset.columns,
            train_dataset.labels,
            train_dataset.edge_types,
        ))
    random.shuffle(tr_data)
    features_train, rows_train, cols_train, ys_train, edge_types_train = zip(
        *tr_data)

    features_train = list(features_train)
    rows_train = list(rows_train)
    cols_train = list(cols_train)
    ys_train = np.array(ys_train)
    edge_types_train = list(edge_types_train)

    for i in range(0, len(features_train), batch_size):
        b_features, b_rows, b_cols, b_ys, b_edges = batch_e(
            features_train[i:i + batch_size],
            rows_train[i:i + batch_size],
            cols_train[i:i + batch_size],
            ys_train[i:i + batch_size],
            edge_types_train[i:i + batch_size],
        )

        trained_params, trained_opt_state, curr_loss = train(
            loss_val_gr,
            opt_update,
            trained_params,
            trained_opt_state,
            b_features,
            b_rows,
            b_cols,
            b_ys,
            b_edges,
        )

        accs = model.accuracy(
            trained_params,
            b_features,
            b_rows,
            b_cols,
            b_ys,
            b_edges,
        )
        print(datetime.datetime.now(),
              f"Iteration {i:4d} | Batch loss {curr_loss:.6f}",
              f"Batch accuracy {accs:.2f}")

    print(datetime.datetime.now(), f"Epoch {ep:2d} completed!")

    # Calculate accuracy across full dataset once per epoch
    print(datetime.datetime.now(), f"Epoch {ep:2d}       | ", end="")
    print_accuracies(model,trained_params, test_dataset, train_dataset)